# OD Visualization Template (Python + GeoPandas + Plotly)
本 Notebook 提供一个可复用的 **交通 OD 分析与可视化** 模板。

In [1]:
import pandas as pd, geopandas as gpd, numpy as np
from shapely.geometry import LineString
import plotly.graph_objects as go
from pathlib import Path

DATA_DIR = Path("../data")
zones = pd.read_csv(DATA_DIR / "zones_centroids.csv")
od = pd.read_csv(DATA_DIR / "od_table.csv")

zones_gdf = gpd.GeoDataFrame(zones, geometry=gpd.points_from_xy(zones.lon, zones.lat), crs="EPSG:4326")


In [2]:
# 构造弯曲连线（简易二次贝塞尔）
import numpy as np
from shapely.geometry import LineString

def get_point(zone_id):
    r = zones_gdf.loc[zones_gdf.zone_id == zone_id]
    return r.geometry.values[0]

def quad_curve(p0, p1, strength=0.3, n=25):
    x0, y0 = p0.x, p0.y
    x1, y1 = p1.x, p1.y
    mx, my = (x0+x1)/2, (y0+y1)/2
    nx, ny = (y1-y0), -(x1-x0)
    L = (nx**2+ny**2)**0.5 or 1.0
    nx, ny = nx/L, ny/L
    cx, cy = mx + strength*nx*max(abs(x1-x0), abs(y1-y0)), my + strength*ny*max(abs(x1-x0), abs(y1-y0))
    ts = np.linspace(0,1,n)
    pts = [((1-t)**2*x0 + 2*(1-t)*t*cx + t**2*x1, (1-t)**2*y0 + 2*(1-t)*t*cy + t**2*y1) for t in ts]
    return LineString(pts)

od['geometry'] = od.apply(lambda r: quad_curve(get_point(r.origin_zone), get_point(r.dest_zone)), axis=1)
gdf_od = gpd.GeoDataFrame(od, geometry='geometry', crs='EPSG:4326')
gdf_top = gdf_od.sort_values('flow_count', ascending=False).head(20)


In [3]:
pip install Plotly -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nbformat>=4.2 -q


Note: you may need to restart the kernel to use updated packages.


In [5]:
# 交互式地图（需要 Plotly）
center_lon, center_lat = zones['lon'].mean(), zones['lat'].mean()

fig = go.Figure()
for _, r in gdf_top.iterrows():
    xs, ys = r.geometry.xy          # xs/ys 是 array('d', ...)
    lon = list(xs)                  # 转成 list
    lat = list(ys)
    fig.add_trace(go.Scattermapbox(
        lon=lon, lat=lat, mode='lines',
        line=dict(width=max(1, r.flow_count/40)),
        hoverinfo='text',
        hovertext=f"{int(r.origin_zone)}→{int(r.dest_zone)} | {int(r.flow_count)}",
        showlegend=False
    ))
fig.add_trace(go.Scattermapbox(
    lon=zones['lon'], lat=zones['lat'], mode='markers+text',
    text=zones['name'], textposition='top center', marker=dict(size=10)
))
fig.update_layout(mapbox_style='open-street-map',
                  mapbox_zoom=11, mapbox_center={'lon':center_lon,'lat':center_lat},
                  margin=dict(l=10,r=10,t=30,b=10),
                  title='Top-20 OD Flows')
fig.show()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_38036\2251227501.py:9: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\Administrator\AppData\Local\Temp\ipykernel_38036\2251227501.py:16: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [7]:
# OD 矩阵热力图（Plotly）
import plotly.figure_factory as ff

pivot = od.pivot_table(index='origin_zone',
                       columns='dest_zone',
                       values='flow_count',
                       aggfunc='sum',
                       fill_value=0)

z = pivot.values
x = pivot.columns.astype(str).tolist()   # 关键：转成 list
y = pivot.index.astype(str).tolist()     # 关键：转成 list

# 可选：显示格内数值
ann = [[str(v) for v in row] for row in z]

fig2 = ff.create_annotated_heatmap(z, x=x, y=y,
                                   annotation_text=ann,   # 想隐藏可去掉这一行
                                   colorscale='Viridis',
                                   showscale=True)
fig2.update_layout(title='OD Matrix Heatmap',
                   xaxis_title='Dest',
                   yaxis_title='Origin')
fig2.show()